In [26]:
import cv2
import numpy as np
from sklearn.metrics import pairwise
%matplotlib inline


In [27]:
background = None
accumulated_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_left = 300
roi_right  = 600

In [28]:
def accumulated_sum(frame, accumulated_weight):
    global background
    if frame is None:
        return None
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)
    
    

In [29]:
def segment( frame, threshold = 25):
    diff = cv2.absdiff(background.astype("uint8"), frame)
    threshold_img = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    image, contours, hierarchy = cv2.findContours(threshold_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours ==0:
        return None
    else: 
        hand_segment = max(contours, key=cv2.contourArea)
        return (threshold_img, hand_segment)
    

In [30]:
def count_finger(threshold_img, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    print(conv_hull)
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    cx = (right[0]+left[0])//2
    cy = (top[1]+bottom[1])//2
    distance = pairwise.euclidean_distances((cx,cy), y=[left, right, top, bottom])[0]
    print(distance)
    max_distance = distance.max()
    radius = int(0.9*max_distance)
    circumference = 2*(np.pi)*radius
    circular_roi = np.zeros(threshold.shape[:2], dtype="uint8")
    cv2.circle(circular_roi, (cx,cy), radius, 255, 10)
    circular_roi = cv2.bitwise_and(threshold_img, threshold_img, mask=circular_roi)
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    count = 0
    for cnt in contours:
        (x,y,wh,h) = cv2.boundingRect(cnt)
        out_of_wrist = ((cy+(cy*.25)) >(y+h))
        limit_point = ((circumference *0.25)>cnt.shape[0])
        if out_of_wrist and limit_point:
            count +=1
    return count 

In [31]:
cam = cv2.VideoCapture(0)
num_frames = 0
while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame,1)
    frame_copy = frame.copy()
    roi = frame_copy[roi_top:roi_bottom, roi_right:roi_left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7,7), 0)
    if num_frames <60:
        accumulated_sum(gray, accumulated_weight)
        if num_frames <=59:
            cv2.putText(frame_copy, "Wait getting the background", (200,400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow("fingercount", frame_copy)
    else:
        hand = segment(gray)
        if hand is not None:
            threshold_img, hand_segment = hand
            cv2.drawContours(frame_copy, [hand_segment+(roi_right, roi_top)], -1, (255,0,0), 1)
            fingers = count_finger(threshold_img, hand_segment)
            cv2.putText(cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2) )
            cv2.imshow( "threshold", threshold_img)
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
    num_frames +=1
      # Display the frame with segmented hand
    cv2.imshow("Finger Count", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break
cam.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'astype'